In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
pip install numpy simplekml

Note: you may need to restart the kernel to use updated packages.


In [18]:
import argparse
from datetime import datetime, timedelta, timezone
import numpy as np
import simplekml
from math import radians, sin, degrees, cos, atan2, sqrt
import random

In [19]:
# constants
R_EARTH = 6378.137  # km (earth radius)
MU = 3.986004418e5  # km³/s², Earth's GM

In [20]:
"""
Computes the great‐circle distance between two latitude/longitude points on a (spherical) Earth. 
Based on haversine formula
"""
def gc_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    fi1, fi2, dg = map(radians, (lat1, lat2, lon2 - lon1))
    a = sin((fi2 - fi1) / 2) ** 2 + cos(fi1) * cos(fi2) * sin(dg / 2) ** 2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

In [21]:
"""
Computes the initial bearing (azimuth) from the point 
(lat1,lon1) to the point (lat2,lon2) on a spherical Earth.
"""
def bearing(lat1, lon1, lat2, lon2):
    fi1, fi2 = np.radians([lat1, lat2])
    dg = np.radians(lon2 - lon1)
    x = np.sin(dg) * np.cos(fi2)
    y = np.cos(fi1) * np.sin(fi2) - np.sin(fi1) * np.cos(fi2) * np.cos(dg)
    brng = np.degrees(np.arctan2(x, y))
    return (brng + 360) % 360  # wrap into 0‥360

In [22]:
"""
circular‐orbit → ground track converter.
approximates where the satellite would be directly overhead on Earth’s surface at that point in its orbit
"""
def eci_to_llh(θ):
    lat = np.arcsin(np.sin(inc_rad) * np.sin(θ))
    # Convert θ→degrees and wrap into [–180, +180]:
    lon_deg = (np.rad2deg(θ) + 180) % 360 - 180
    return np.rad2deg(lat), lon_deg

In [23]:
"""
Return lat,lon reached by starting at (lat,lon) and going
'bearing_deg' for 'dist_km' km on a sphere.
"""
def gc_destination(lat, lon, bearing_deg, dist_km):
    R = 6371.0
    fi1, g1, ang_displacement = map(radians, (lat, lon, bearing_deg))
    d = dist_km / R
    fi2 = sin(fi1) * cos(d) + cos(fi1) * sin(d) * cos(ang_displacement)
    fi2 = atan2(fi2, sqrt(1 - fi2 ** 2))  # lat
    g2 = g1 + atan2(sin(ang_displacement) * sin(d) * cos(fi1),
                    cos(d) - sin(fi1) * sin(fi2))  # lon
    return degrees(fi2), (degrees(g2) + 540) % 360 - 180

In [24]:
def generate_evenly_spaced_targets(track_latlon, n_points = 10,
                                   lateral_km = 0):
    """
    Pick n_points along the given track_latlon so that each target
    lies roughly 'total_track_length / n_points' km apart.

    1. Compute the cumulative distance along track_latlon (in km).
    2. Let spacing = (total_length / n_points).
    3. Select the first sample whose cumulative_dist >= k * spacing, for k = 1..n_points.
    4. (Optionally) offset each selected point laterally by up to lateral_km in a random direction.
       If lateral_km == 0, no offset is applied and the target lies exactly on the computed ground track.

    Returns a list of (lat, lon) for each of the n_points targets.
    """

    # build a list of cumulative distances along track_latlon
    cumdist = [0.0]
    for i in range(1, len(track_latlon)):
        latA, lonA = track_latlon[i-1]
        latB, lonB = track_latlon[i]
        d = gc_distance(latA, lonA, latB, lonB)
        cumdist.append(cumdist[-1] + d)

    total_length = cumdist[-1]
    if total_length <= 0:
        raise ValueError("Track is too short (zero total length).")

    # compute how far apart each target should be
    spacing_km = total_length / n_points

    out = []
    next_threshold = spacing_km
    idx = 0

    # for each of the n_points, find the appropriate index
    for _ in range(n_points):
        while idx < len(cumdist) and cumdist[idx] < next_threshold:
            idx += 1

        # If we ran off the end, clamp to the last sample
        if idx >= len(cumdist):
            idx = len(cumdist) - 1

        base_lat, base_lon = track_latlon[idx]

        if lateral_km > 0:
            # if lateral_km > 0, randomly offset that base point by up to lateral_km
            bearing_rand = random.uniform(0, 360)
            tgt_lat, tgt_lon = gc_destination(base_lat, base_lon, bearing_rand, lateral_km)
        else:
            # no offset: target is exactly on the ground track
            tgt_lat, tgt_lon = base_lat, base_lon

        out.append((tgt_lat, tgt_lon))

        # increase the threshold for the next target
        next_threshold += spacing_km

    return out

In [25]:
import sys
import math
from datetime import datetime, timedelta, timezone
from xml.etree import ElementTree as ET
import xml.etree.ElementTree as ET

In [26]:
"""
  Generate a closed circle polygon of latitude/longitude points.
    Returns:
    A list of (longitude, latitude, altitude) tuples forming a closed 
    polygon. Altitude is set to 0 for all points.
"""

def create_circle_coords(lat_c, lon_c, radius_km, n_pts=36):
    pts=[]
    for b in np.linspace(0,360,n_pts,endpoint=False):
        lati,loni = gc_destination(lat_c, lon_c, b, radius_km)
        pts.append((loni, lati, 0))
    pts.append(pts[0])
    return pts

In [27]:
parser = argparse.ArgumentParser(description="Create animated satellite track KML.")
parser.add_argument("--alt", type=float, default=250, help="Altitude above mean sea level in km")
parser.add_argument("--inc", type=float, default=0, help="Orbital inclination in degrees")
parser.add_argument("--period", type=float, default=None, help="Orbital period in seconds "
                                                               "(leave blank for Keplerian)")
parser.add_argument("--step", type=float, default=10, help="Time step in seconds")
parser.add_argument("--hours", type=float, default=2, help="Total simulated hours")
if 'ipykernel' in sys.modules:
    args = parser.parse_args(args=['--alt', '350', '--inc', '53', '--hours', '1.7'])
else:
    args = parser.parse_args()

In [28]:
# Orbit and visualization parameter initialization
h = args.alt
r = R_EARTH + h
inc_rad = np.deg2rad(args.inc)
scale_factor = 0.2
min_radius   = 50

# If user didn’t specify a period, compute it for a circular orbit
if args.period is None:
    args.period   = 2 * math.pi * math.sqrt(r**3 / MU)
ang_vel = 2 * math.pi / args.period

# Angular velocity of the satellite (radians per second)
ang_velocity = 2 * np.pi / args.period  # angular velocity of the satellite (radians per sec)

In [29]:
start_utc = datetime.now(timezone.utc)
t_span    = np.arange(0, args.hours*3600 + args.step, args.step)

#Track
kml = simplekml.Kml()
track = kml.newgxtrack(name=f"Satellite @ {h}km, inc {args.inc}°")
when_list, coord_list = [], []
# For each time t in t_span, compute the satellite’s argument of latitude
for t in t_span:
    θ = ang_vel * t
    lat, lon = eci_to_llh(θ) # Convert that to geographic lat/lon via eci_to_llh
    when_list.append((start_utc + timedelta(seconds=float(t))).isoformat())
    coord_list.append((lon, lat, h*1000))
track.newwhen(when_list)
track.newgxcoord(coord_list)
track.altitudemode = simplekml.AltitudeMode.absolute
track.style.iconstyle.icon.href = "http://maps.google.com/mapfiles/kml/shapes/satellite.png"

tour = kml.newgxtour(name="Orbit")
playlist = tour.newgxplaylist()


In [30]:
# Targets
track_samples = [eci_to_llh(ang_vel*t) for t in t_span]
targets       = generate_evenly_spaced_targets(track_samples, n_points=10, lateral_km=0) #Generate targets
targets_named = [(lat, lon, f"Target #{i+1}") for i,(lat,lon) in enumerate(targets)]

target_pm_strs = []
# Place a yellow point marker for each target
for lat, lon, name in targets_named:
    pm = kml.newpoint(name=name, coords=[(lon, lat)])
    pm.style.iconstyle.color = simplekml.Color.yellow 
    pm.style.iconstyle.scale = 0.8
    
    target_pm_strs.append(f"""
    <Placemark>
      <name>{name}</name>
      <Point><coordinates>{lon},{lat},0</coordinates></Point>
    </Placemark>""")

# circlePlacemark, create an initial static circle placemark
tgt_lat, tgt_lon, _ = targets_named[0]
init_dist = gc_distance(*eci_to_llh(0), tgt_lat, tgt_lon)
radius0   = init_dist * scale_factor # Compute its first radius based on initial satellite target distance
coords0   = create_circle_coords(tgt_lat, tgt_lon, radius0) # Use `create_circle_coords` to build the polygon coordinates

# gives a handle so we can update it later via <gx:AnimatedUpdate>
circle_kml = f"""
    <Placemark id="circlePlacemark">
      <name>Dynamic Circle</name>
      <Style>
        <LineStyle><color>ff0000ff</color><width>2</width></LineStyle>
        <PolyStyle><color>7f0000ff</color></PolyStyle>
      </Style>
      <Polygon>
        <altitudeMode>clampToGround</altitudeMode>
        <outerBoundaryIs><LinearRing>
          <coordinates>{coords0}</coordinates>
        </LinearRing></outerBoundaryIs>
      </Polygon>
    </Placemark>"""

# Header opens `<Document>`, injects target placemarks and the circle
header = f"""<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2"
     xmlns:gx="http://www.google.com/kml/ext/2.2">
<Document>
  <name>Orbit Tour</name>
  {''.join(target_pm_strs)}
  {circle_kml}
  <gx:Tour>
    <name>Orbit fly-along</name>
    <gx:Playlist>"""

# Footer closes the `<gx:Playlist>` and document tags
footer = """
    </gx:Playlist>
  </gx:Tour>
</Document>
</kml>"""

body = []
step_cam = 5
idx_step = int(step_cam/args.step)
range_km = 650

camera_mode = "CRUISE"
last_heading = 0
last_tilt = 15
active_target = None

for i in range(len(t_span)):
    θ_now = ang_vel * t_span[i]
    sat_lat, sat_lon = eci_to_llh(θ_now)

    # find the closest target
    dists = [(idx, gc_distance(sat_lat, sat_lon, lt, ln), lt, ln)
             for idx, (lt, ln, _) in enumerate(targets_named)]
    idx_min, min_dist, tgt_lat, tgt_lon = min(dists, key=lambda x: x[1])

    # range check 
    if min_dist <= range_km:
        # entered to the trigger zone of some target
        if active_target != idx_min:
            # entered to the trigger radius of a new target
            active_target = idx_min
            camera_mode = "ON_TARGET"
            hdg = bearing(sat_lat, sat_lon, tgt_lat, tgt_lon)
            tilt = math.degrees(math.atan2(min_dist, h))
            last_heading = hdg
            last_tilt = tilt
        else:
            # still with the same target, keep following
            hdg = bearing(sat_lat, sat_lon, tgt_lat, tgt_lon)
            tilt = math.degrees(math.atan2(min_dist, h))
            last_heading = hdg
            last_tilt = tilt
    else:
        # no target in range
        if camera_mode == "ON_TARGET":
            # exits range and goes into mode OFF, preserving last heading/tilt
            camera_mode = "OFF"
            active_target = None
        hdg = last_heading
        tilt = last_tilt
    # Moves the camera smoothly to “look at” the satellite’s current lat/lon/altitude.
    # heading and tilt point the camera toward the active target.
    body.append(f"""
      <gx:FlyTo>
        <gx:duration>{step_cam}</gx:duration>
        <gx:flyToMode>smooth</gx:flyToMode>
        <LookAt>
          <longitude>{sat_lon}</longitude>
          <latitude>{sat_lat}</latitude>
          <altitude>{h*1000}</altitude>
          <heading>{hdg:.1f}</heading>
          <tilt>{tilt:.1f}</tilt>
          <range>{h*1000}</range>
          <altitudeMode>absolute</altitudeMode>
        </LookAt>
      </gx:FlyTo>""")

    radius = min_dist * scale_factor
    radius = max(radius, min_radius)
    new_coords = create_circle_coords(tgt_lat, tgt_lon, radius)
    coords_str  = " ".join(f"{lo},{la},0" for lo,la,_ in new_coords)

    # Calls create_circle_coords to get new boundary points and embeds them into an AnimatedUpdate.
    body.append(f"""
      <gx:AnimatedUpdate>
        <gx:duration>{step_cam}</gx:duration>
        <Update><Change>
          <Placemark targetId="circlePlacemark">
            <Polygon>
              <outerBoundaryIs><LinearRing>
                <coordinates>{coords_str}</coordinates>
              </LinearRing></outerBoundaryIs>
            </Polygon>
          </Placemark>
        </Change></Update>
      </gx:AnimatedUpdate>""")


# Write out final KML file
full_kml = header + "\n".join(body) + footer
with open(f"orbit_with_dynamic_circle{args.inc:.0f}deg.kml","w",encoding="utf-8") as f:
    f.write(full_kml)

print("Saved KML with a single shrinking circle around the target.")

Saved KML with a single shrinking circle around the target.
